# Importing Libraries & datasets

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_excel("..\dataset\ProductA.xlsx")
df2 = pd.read_excel("..\dataset\ProductA_google_clicks.xlsx")
df3 = pd.read_excel("..\dataset\ProductA_fb_impressions.xlsx")

# Data Preprocessing

### Merging datasets

In [3]:
df4 = pd.merge(df1, df2, on="Day Index", how="left")        # left joined
df = pd.merge(df4, df3, on="Day Index", how="left")

In [4]:
df.head()

Day Index  Quantity  Clicks  Impressions
0 2021-12-01        14     445          620
1 2021-12-02        10     433          890
2 2021-12-03        13     424          851
3 2021-12-04        22     427          881
4 2021-12-05        33     451          678

### Descriptive Statistics

In [5]:
df.shape

(212, 4)

| Rows | Columns |
|------|---------|
| 212  | 4       |


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Day Index    212 non-null    datetime64[ns]
 1   Quantity     212 non-null    int64         
 2   Clicks       212 non-null    int64         
 3   Impressions  212 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 6.8 KB


In [7]:
df.describe()

Day Index    Quantity     Clicks  Impressions
count                  212  212.000000  212.00000   212.000000
mean   2022-03-16 12:00:00   17.542453  358.20283  1531.702830
min    2021-12-01 00:00:00    5.000000   60.00000   620.000000
25%    2022-01-22 18:00:00   13.750000  154.00000  1209.750000
50%    2022-03-16 12:00:00   17.000000  402.50000  1460.000000
75%    2022-05-08 06:00:00   21.000000  492.75000  1862.000000
max    2022-06-30 00:00:00   38.000000  726.00000  2707.000000
std                    NaN    5.790423  164.52160   440.660783

### Missing values - Imputation

In [8]:
print("NULL Values:")
df.isnull().sum()

NULL Values:


Day Index      0
Quantity       0
Clicks         0
Impressions    0
dtype: int64

Hence, there are no NULL values. So, No imputation needed.

### Dropping Duplicates

In [9]:
df.shape

(212, 4)

In [10]:
df.drop_duplicates()

Day Index  Quantity  Clicks  Impressions
0   2021-12-01        14     445          620
1   2021-12-02        10     433          890
2   2021-12-03        13     424          851
3   2021-12-04        22     427          881
4   2021-12-05        33     451          678
..         ...       ...     ...          ...
207 2022-06-26        13     114         1218
208 2022-06-27        16     106         1311
209 2022-06-28        13     109          885
210 2022-06-29        16      60         1000
211 2022-06-30         8      65          875

[212 rows x 4 columns]

No row dropped after drop_duplicates() ! Hence, No duplicates found! 


### Analysing Outliers

In [11]:
plt.figure()
df.boxplot()
plt.show()

In [12]:
def find_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers

print("Outliers in each dependent attribute:")
print(f"Quantity:\t {len(find_outliers_IQR(df['Quantity']))}")
print(f"Clicks:\t\t {len(find_outliers_IQR(df['Clicks']))}")
print(f"Impressions:\t {len(find_outliers_IQR(df['Impressions']))}")

Outliers in each dependent attribute:
Quantity:	 5
Clicks:		 0
Impressions:	 0


So, there are 5 outliers in the *Quantity* attribute

In [13]:
print(f"Quantity:\n{find_outliers_IQR(df['Quantity'])}")

Quantity:
4      33
9      38
78     38
113    32
142    36
Name: Quantity, dtype: int64


***

### Replacing Outliers with 97th percentile value

In [14]:
percentile_95_value = df["Quantity"].quantile(0.97)
print(percentile_95_value)

30.669999999999987


In [15]:
def impute_outliers_97th_percentile(series):
    q1, q3 = series.quantile([0.25, 0.75])
    IQR = q3 - q1
    lower_bound, upper_bound = q1 - 1.5 * IQR, q3 + 1.5 * IQR
    percentile_95_value = series.quantile(0.97)
    return series.apply(lambda x: percentile_95_value if x < lower_bound or x > upper_bound else x)

In [16]:
df['imputed_Quantity'] = impute_outliers_97th_percentile(df['Quantity'])

In [17]:
print("Outliers in each dependent attribute:")
print(f"Quantity:\t {len(find_outliers_IQR(df['imputed_Quantity']))}")

Outliers in each dependent attribute:
Quantity:	 0


Hence, the outliers are replaced by 97th percentile values.

# Feature Engineering

In [18]:
df.head()

Day Index  Quantity  Clicks  Impressions  imputed_Quantity
0 2021-12-01        14     445          620             14.00
1 2021-12-02        10     433          890             10.00
2 2021-12-03        13     424          851             13.00
3 2021-12-04        22     427          881             22.00
4 2021-12-05        33     451          678             30.67

Imputed_Quantity signifies the quantity column where outliers are set on 97th percentile.

In [19]:
df['imputed_Quantity'] = impute_outliers_97th_percentile(df['Quantity'])

Log Tranformed attributes

In [20]:
df['log_Quantity'] = np.log1p(df['Quantity'])
df['log_Clicks'] = np.log1p(df['Clicks'])
df['log_Impressions'] = np.log1p(df['Impressions'])

Extract Month and Year

In [21]:
df['YearMonth'] = df['Day Index'].dt.to_period('M')

Extract Month Name

In [22]:
df['MonthName'] = df['YearMonth'].dt.strftime('%B')
month_order = ['December', 'January', 'February', 'March', 'April', 'May', 'June']  
df['MonthName'] = pd.Categorical(df['MonthName'], categories=month_order, ordered=True)

Extract day of the Week

In [23]:
df['DayOfWeek'] = df['Day Index'].dt.day_name()
week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']  
df['DayOfWeek'] = pd.Categorical(df['DayOfWeek'], categories=week_order, ordered=True)

Extract week of month

In [24]:
def week_of_month(date):
    first_day = date.replace(day=1)
    dom = date.day
    adjusted_dom = dom + first_day.weekday()
    return int(np.ceil(adjusted_dom / 7.0))

df['Week_of_Month'] = df['Day Index'].apply(week_of_month)

Mapping weekdays and months into separate columns 

In [25]:
# Map weekdays and months to 1 where applicable
df.loc[df['DayOfWeek'] == 'Monday', 'Weekday_Monday'] = 1
df.loc[df['DayOfWeek'] == 'Tuesday', 'Weekday_Tuesday'] = 1
df.loc[df['DayOfWeek'] == 'Wednesday', 'Weekday_Wednesday'] = 1
df.loc[df['DayOfWeek'] == 'Thursday', 'Weekday_Thursday'] = 1
df.loc[df['DayOfWeek'] == 'Friday', 'Weekday_Friday'] = 1
df.loc[df['DayOfWeek'] == 'Saturday', 'Weekday_Saturday'] = 1
df.loc[df['DayOfWeek'] == 'Sunday', 'Weekday_Sunday'] = 1

df.loc[df['MonthName'] == 'January', 'Month_January'] = 1
df.loc[df['MonthName'] == 'February', 'Month_February'] = 1
df.loc[df['MonthName'] == 'March', 'Month_March'] = 1
df.loc[df['MonthName'] == 'April', 'Month_April'] = 1
df.loc[df['MonthName'] == 'May', 'Month_May'] = 1
df.loc[df['MonthName'] == 'June', 'Month_June'] = 1
df.loc[df['MonthName'] == 'July', 'Month_July'] = 1
df.loc[df['MonthName'] == 'August', 'Month_August'] = 1
df.loc[df['MonthName'] == 'September', 'Month_September'] = 1
df.loc[df['MonthName'] == 'October', 'Month_October'] = 1
df.loc[df['MonthName'] == 'November', 'Month_November'] = 1
df.loc[df['MonthName'] == 'December', 'Month_December'] = 1

replacing any NaN value with zero


In [26]:
weekday_columns = ['Weekday_Monday', 'Weekday_Tuesday', 'Weekday_Wednesday', 'Weekday_Thursday',
                   'Weekday_Friday', 'Weekday_Saturday', 'Weekday_Sunday']
month_columns = ['Month_January', 'Month_February', 'Month_March', 'Month_April', 'Month_May',
                 'Month_June', 'Month_July', 'Month_August', 'Month_September', 'Month_October',
                 'Month_November', 'Month_December']

# Fill NaNs with 0 for weekday and month columns
df[weekday_columns] = df[weekday_columns].fillna(0)
df[month_columns] = df[month_columns].fillna(0)


final data

In [27]:
df.columns

Index(['Day Index', 'Quantity', 'Clicks', 'Impressions', 'imputed_Quantity',
       'log_Quantity', 'log_Clicks', 'log_Impressions', 'YearMonth',
       'MonthName', 'DayOfWeek', 'Week_of_Month', 'Weekday_Monday',
       'Weekday_Tuesday', 'Weekday_Wednesday', 'Weekday_Thursday',
       'Weekday_Friday', 'Weekday_Saturday', 'Weekday_Sunday', 'Month_January',
       'Month_February', 'Month_March', 'Month_April', 'Month_May',
       'Month_June', 'Month_July', 'Month_August', 'Month_September',
       'Month_October', 'Month_November', 'Month_December'],
      dtype='object')

In [28]:
df.head()

Day Index  Quantity  Clicks  Impressions  imputed_Quantity  log_Quantity  \
0 2021-12-01        14     445          620             14.00      2.708050   
1 2021-12-02        10     433          890             10.00      2.397895   
2 2021-12-03        13     424          851             13.00      2.639057   
3 2021-12-04        22     427          881             22.00      3.135494   
4 2021-12-05        33     451          678             30.67      3.526361   

   log_Clicks  log_Impressions YearMonth MonthName  ... Month_March  \
0    6.100319         6.431331   2021-12  December  ...         0.0   
1    6.073045         6.792344   2021-12  December  ...         0.0   
2    6.052089         6.747587   2021-12  December  ...         0.0   
3    6.059123         6.782192   2021-12  December  ...         0.0   
4    6.113682         6.520621   2021-12  December  ...         0.0   

   Month_April  Month_May  Month_June  Month_July  Month_August  \
0          0.0        0.0         0.0         0.0           0.0   
1          0.0        0.0         0.0         0.0           0.0   
2          0.0        0.0         0.0         0.0           0.0   
3          0.0        0.0         0.0         0.0           0.0   
4          0.0        0.0         0.0         0.0           0.0   

   Month_September  Month_October  Month_November  Month_December  
0              0.0            0.0             0.0             1.0  
1              0.0            0.0             0.0             1.0  
2              0.0            0.0             0.0             1.0  
3              0.0            0.0             0.0             1.0  
4              0.0            0.0             0.0             1.0  

[5 rows x 31 columns]

exporting the final dataset for further analysis

In [29]:
df.to_excel('../final_df.xlsx', index=False)   